# BFGS (Broyden–Fletcher–Goldfarb–Shanno)

In [ ]:

import numpy as np
import numpy.linalg as ln
import scipy as sp
import scipy.optimize
import numdifftools as nd
# Objective function
def f(x):
    return 4*(x[0]**2) + (x[0] * x[1])+ x[1]**2 - x[0] - 3*x[1] +7

def bfgs_method(f, x0, maxiter=None, epsi=10e-12):
    # Derivative
    fprime=nd.Gradient(f)
    print(fprime)
    if maxiter is None:
        maxiter = len(x0) * 200
    # initial values
    k = 0
    gfk = fprime(x0)
    N = len(x0)
    I = np.eye(N, dtype=int)
    Hk = I
    xk = x0
    while ln.norm(gfk) > epsi and k < maxiter:
        pk = -np.dot(Hk, gfk)
        line_search = sp.optimize.line_search(f, fprime, xk, pk)
        alpha_k = line_search[0]
        xkp1 = xk + alpha_k * pk
        sk = xkp1 - xk #delta X
        xk = xkp1
        gfkp1 = fprime(xkp1)
        yk = gfkp1 - gfk  #delta gradient
        gfk = gfkp1
        k += 1
        ro = 1.0 / (np.dot(yk, sk))
        A1 = I - ro * sk[:, np.newaxis] * yk[np.newaxis, :]
        A2 = I - ro * yk[:, np.newaxis] * sk[np.newaxis, :]
        Hk = np.dot(A1, np.dot(Hk, A2)) + (ro * sk[:, np.newaxis] * sk[np.newaxis, :])
    return (xk, k)
result, k = bfgs_method(f, np.array([0,0]))
print('The solution of bfgs algorithm is %s' % (result))

The solution of bfgs algorithm is [-0.06666667  1.53333333]


# Steepest Descent Algorithm for General Functions


In [ ]:
#!pip install numdifftools
import numpy as np
import numdifftools as nd
from numpy.linalg import norm

def Steepest_Descent(x,eps=10e-8):

    i = 0 
    while True:
        grad = nd.Gradient(func)(x[i]) 
        grad_norm = norm(grad,2)      
        if grad_norm < eps:            
            print(f"minimum point {x[i]} is obtained at iteration {i}")
            break
        _dir = -grad                   
        H = nd.Hessian(func)(x[i])     
        alpha = (_dir.T @ _dir) / (_dir.T @ H @ _dir) 
               
        new_x = x[i] + alpha * _dir   
        x.append(new_x)               
       
        i+=1                         
    
    return

def func(x): 
    return ( ((x[0] - 2)**2)/8 + ((x[1]-3)**2)/12  ) 

x = []                  
x.append([1,1])      
Steepest_Descent(x)  

minimum point [1.99999993 2.99999986] is obtained at iteration 10


#Newton's Minimization

In [ ]:
import numpy as np
from sympy import *
def hessian(f,xi):
  symb=[k for k in [x,y,z,v] if f.count(k)!=0]
  n=len(symb)
  M=np.zeros((n,n))
  for p in range(0,n):
    for j in range(0,n):
      holder=diff(f,symb[p],symb[j])
      for q in range(0,n):
        holder=holder.subs(symb[q],xi[q])
      M[p][j]=holder
  M=np.linalg.inv(M)
  return M
def grad(f,xi):
  symb=[k for k in [x,y,z,v] if f.count(k)!=0]
  n=len(symb)
  M=np.zeros(n)
  for p in range(0,n):
    holder=diff(f,symb[p])
    for q in range(0,n):
      holder=holder.subs(symb[q],xi[q])
    M[p]=holder
  M=M.reshape(n,1)
  return M
def Newton_min(f,tol=10e-5):
  symb=[k for k in [x,y,z,v] if f.count(k)!=0]
  n=len(symb)
  x0=np.array([28,10,6]).reshape(n,1)# initial value
  i=0
  while True:
    hess=hessian(f,x0)        
    gradient=grad(f,x0)       
    xk=x0-np.dot(hess,gradient)  
    error=np.linalg.norm(x0-xk)
    x0=xk
    if i>200:
      break
    if error<tol:
      break
    i+=1
  return x0,i


x,y,z,v=symbols('x y z v')       
f=(x-30)**4+2*(y-12)**2+80*(z-5)**6           
X,i=Newton_min(f)
print("Iteration ",i)
print("The minima point is : ",X)

Iteration  35
The minima point is :  [[29.99999908]
 [12.        ]
 [ 5.00032452]]


# Steepest Descent for Quadatic Form

In [ ]:
def f(x,Q,b,c):
    return (0.5*(x.T).dot(Q.dot(x))+(x.T).dot(b)+c)

def grad_f(x,Q,b,c):
    return (Q.dot(x)+b)

def SteepestDescent(x,Q,b,c):
    while True:
        temp=x
        g=grad_f(x,Q,b,c)
        alpha=((g.T).dot(g))/((g.T).dot(Q.dot(g)))
        x=x-alpha*grad_f(x,Q,b,c)
        if (np.linalg.norm(x-temp)<1e-12):
            break
    return x

In [ ]:
x = np.array([[0.0],[0.0]])
Q = np.array([[2.0, 5.0], [5.0, 15.0]])
b = np.array([[1.0], [2.0]])  
c = 8
x=SteepestDescent(x,Q,b,c)
print("The optimum point is : ",x)

The optimum point is :  [[-1. ]
 [ 0.2]]


# Conjugate Gradient method

In [ ]:
import numpy as np
A=np.array([[8,1],[1,2]])
#print(Q)
b=np.array([1,3])
x=np.array([0,0])

def CJ(A,b,x):
    r = b - np.dot(A, x)
    p = r
    rsold = np.dot(np.transpose(r), r)
    for i in range(len(b)):
        Ap = np.dot(A, p)
        alpha = rsold / np.dot(np.transpose(p), Ap)
        x = x + np.dot(alpha, p)
        r = r - np.dot(alpha, Ap)
        rsnew = np.dot(np.transpose(r), r)
        if np.sqrt(rsnew) < 1e-12:
            break
        p = r + (rsnew/rsold)*p
        rsold = rsnew
        #print(x)
    return x

x=CJ(A,b,x)
print("solution is ",x)

solution is  [-0.06666667  1.53333333]


### Rank One

In [ ]:
import numpy as np
import numpy.linalg as ln
import scipy as sp
import scipy.optimize
import numdifftools as nd

# Objective function
def f(x):
    

def rank_one_method(f, x0, maxiter=None, epsi=10e-6):
    fprime=nd.Gradient(f)
    if maxiter is None:
        maxiter = len(x0) * 200
    k = 0
    gfk = fprime(x0)
    N = len(x0)
    I = np.eye(N, dtype=int)
    Hk = I
    xk = x0
    while ln.norm(gfk) > epsi and k < maxiter:
        pk = -np.dot(Hk, gfk)
        line_search = sp.optimize.line_search(f, fprime, xk, pk)
        alpha_k = line_search[0]
        xkp1 = xk + alpha_k * pk
        sk = xkp1 - xk #delta X
        xk = xkp1
        gfkp1 = fprime(xkp1)
        yk = gfkp1 - gfk  #delta gradient
        gfk = gfkp1
        
        u = sk - Hk @ yk
        Hk = Hk + (u @ u.T)/(u.T @ yk)
        
        k += 1
    return (xk, k)


result, k = rank_one_method(f, np.array([0, 0]), 2)

print('minima point: %s' % (result))

minima point: [-0.02268802  1.61892619]


### Rank Two or DFP

In [ ]:
import numpy as np
import numpy.linalg as ln
import scipy as sp
import scipy.optimize
import numdifftools as nd

# Objective function
def f(x):
    return 4*(x[0]**2) + (x[0] * x[1])+ x[1]**2 - x[0] - 3*x[1] +7


def rank_two_method(f, x0, maxiter=None, epsi=10e-2):
   
    fprime=nd.Gradient(f)
    if maxiter is None:
        maxiter = len(x0) * 200

    k = 0
    gfk = fprime(x0)
    N = len(x0)
    I = np.eye(N, dtype=int)
    Hk = I
    xk = x0
    while ln.norm(gfk) > epsi and k < maxiter:
        pk = -np.dot(Hk, gfk)

        line_search = sp.optimize.line_search(f, fprime, xk, pk)
        alpha_k = line_search[0]
        xkp1 = xk + alpha_k * pk
        sk = xkp1 - xk 
        xk = xkp1
        gfkp1 = fprime(xkp1)
        yk = gfkp1 - gfk  
        gfk = gfkp1
        
        u = Hk @ yk
        Hk = Hk + (sk @ sk.T)/(sk.T @ yk) - (u @ u.T)/(yk.T @ u)
        
        k += 1
    return (xk, k)


result, k = rank_two_method(f, np.array([0, 0]), 2)
print(' The soultion is: %s' % (result))

 The soultion is: [-0.07523734  1.50419304]
